In [165]:
#from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreceated sklearn

ERROR: Invalid requirement: '#'


Task1

In [166]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./blood_transfusion.csv"))
frame.describe(include='all') # 5 columns

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,9.506684,5.514706,1378.676471,34.282086,0.237968
std,8.095396,5.839307,1459.826781,24.376714,0.426124
min,0.000000,1.000000,250.000000,2.000000,0.000000
25%,2.750000,2.000000,500.000000,16.000000,0.000000
50%,7.000000,4.000000,1000.000000,28.000000,0.000000
75%,14.000000,7.000000,1750.000000,50.000000,0.000000
max,74.000000,50.000000,12500.000000,98.000000,1.000000


In [167]:
# visualise, understand, story

Task2

In [168]:
preprocessed = frame.copy() 
preprocessed = (preprocessed-preprocessed.min())/(preprocessed.max()-preprocessed.min()) # Normalization
preprocessed.describe()

,months_since_last_donation,total_number_of_donations,total_blood_donated,months_since_first_donation,class
count,748.000000,748.000000,748.000000,748.000000,748.000000
mean,0.128469,0.092137,0.092137,0.336272,0.237968
std,0.109397,0.119170,0.119170,0.253924,0.426124
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.037162,0.020408,0.020408,0.145833,0.000000
50%,0.094595,0.061224,0.061224,0.270833,0.000000
75%,0.189189,0.122449,0.122449,0.500000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


TASK 3

In [169]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train_s, X_test_s, Y_train_s, Y_test_s = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], test_size=0.33, random_state=1) # small test_size
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(
    preprocessed.drop('class', axis=1), preprocessed['class'], test_size=0.66, random_state=1) # large test_size
# random state for reproducibility

Task 4

In [173]:
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score

#5.1 Confusion matrix
def confusionmatrix(predicted_y, real_y): # it works, im confused :)
    """
    This function calculates and prints the confusion matrix.
    :param predicted_y: list: predicted classes for the test data 
    :param real_y: list: real classes of the test data
    :return: prints the confusion matrix, returns nothing
    """
    tp, fp, tn, fn = 0, 0, 0, 0
    for index,_ in enumerate(predicted_y):
        if predicted_y[index] == 1: 
            if predicted_y[index] == real_y[index]: 
                tp += 1
            else:
                fp += 1
        elif predicted_y[index] == 0:
            if predicted_y[index] == real_y[index]: 
                tn += 1
            else:
                fn += 1
    print(f"\t P \t\t N \n T \t {tp} \t\t {tn} \n F \t {fp} \t\t {fn} \n")

def accuracy_checker(y_test, pred, mode="basic", X_test=None, y_predicted=None, knn=False):
    """
    This function checks the accuracy of the predicted classes against the real classes.
    :param X_test: pandas.DataFrame: the test data to be classified
    :param y_predicted: list: predicted classes for the test data
    :param y_test: pandas.Series: the real classes of the test data
    :param pred: predictor class: the method used to classify the test data
    :param mode: string: way to evaluate the results
    :return: prints the results, returns nothing
    """
    num, total = 0, 0
    predicted, true = [], []
    if not knn:
        for index, row in X_test.iterrows():
            p = pred.predict(X_test.loc[[index]])
            q = list(y_test.loc[[index]])
            if p == q:
                num += 1
            total += 1
            predicted.append(p)
            true.append(q)
    else: 
        predicted = y_predicted
        true = list(y_test)
        for index, _ in enumerate(predicted):
            if predicted[index] == true[index]: 
                num += 1
            total += 1
        
    if mode == "basic":
        print(f"Accuracy: {num/total*100:.2f}%")
    elif mode == "classification_report":
        print(classification_report(true, predicted))
    elif mode == "confusion_matrix":
        confusionmatrix(true, predicted)
    elif mode == "fbeta":
        print("average=macro:", fbeta_score(true, predicted, average='macro', beta=0.5))
        print("average=weighted:", fbeta_score(true, predicted, average='weighted', beta=0.5))
        print("average=None:", fbeta_score(true, predicted, average=None, beta=0.5))

# 4.1 KNN
def manhattan_distance(a, b):
    """
    This function calculates the manhattan distance between two datapoints.
    :param a: list: the first datapoint
    :param b: list: the second datapoint
    :return: float: the manhattan distance between the datapoints
    """
    distance = 0
    for index, _ in enumerate(a):
        distance += ((a[index] - b[index]) ** 2) ** 0.5
    return distance      

def KNN_classifier(k, train_x, train_y, test_x):
    """
    This function trains the KNN classifier on the training data and predicts the class of the test data.
    :param k: int: the number of neighbors
    :param train_x: pandas.DataFrame: the train data
    :param train_y: pandas.Series: the corresponding classes of the train data
    :param test_x: pandas.DataFrame: the test data
    :return: list: the predicted classes of the test data
    """
    test_y = list([0 for _ in range(test_x.shape[0])])
    for index, (_,test_row) in enumerate(test_x.iterrows()):
        distances = []
        for _, train_row in train_x.iterrows():
            test_values = list(test_row)  
            train_values = list(train_row)
            distance = manhattan_distance(test_values, train_values)
            distances.append(distance)
        nearest_neighbours = sorted(range(len(distances)), key=lambda i: distances[i])[:k]
        neighbor_labels = train_y.iloc[nearest_neighbours]
        predicted_label = round(neighbor_labels.mean())
        test_y[index] = predicted_label
    return test_y

y_predicted = KNN_classifier(5, X_train_s, Y_train_s, X_test_s)
print("For knn:")
accuracy_checker(Y_test_s, "knn", y_predicted=y_predicted, knn=True)

# 4.2 nbc
from sklearn.naive_bayes import GaussianNB
nbc = GaussianNB()
nbc.fit(X_train_s, Y_train_s)
print("For nbc:")
accuracy_checker(Y_test_s, nbc, X_test=X_test_s)

# 4.3 svc
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
svc = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc.fit(X_train_s, Y_train_s)
print("For svc:")
accuracy_checker(Y_test_s, svc, X_test=X_test_s)

# 4.4 mlp
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=250).fit(X_train_s, Y_train_s)
print("For mlp:")
accuracy_checker(Y_test_s, mlp, X_test=X_test_s)

For knn:
Accuracy: 77.73%
For nbc:
Accuracy: 74.90%
For svc:
Accuracy: 76.11%
For mlp:
Accuracy: 76.92%


In [182]:
Y_predicted_s = KNN_classifier(5, X_train_s, Y_train_s, X_test_s)
Y_predicted_l = KNN_classifier(5, X_train_l, Y_train_l, X_test_l)
preds = [nbc, svc, mlp, "knn"]
modes = ["confusion_matrix", "classification_report", "fbeta"]
for mode in modes:
    for pred in preds:
        if pred == "knn":
            print(f"{mode} for {pred} with small test data size:")
            accuracy_checker(Y_test_s, pred, mode=mode, y_predicted=Y_predicted_s, knn=True)
            print(f"\n{mode} for {pred} with large test data size:")
            accuracy_checker(Y_test_l, pred, mode=mode, y_predicted=Y_predicted_l, knn=True)
            print("-----------------------------------------")
        else:
            pred_string = ""
            if pred == svc: pred_string = "svc"
            elif pred == mlp: pred_string = "mlp"
            elif pred == nbc: pred_string = "nbc"
            print(f"{mode} for {pred_string} with small test data size:")
            accuracy_checker(Y_test_s, pred, mode=mode, X_test=X_test_s)            
            print(f"\n{mode} for {pred_string} with large test data size:")
            accuracy_checker(Y_test_l, pred, mode=mode, X_test=X_test_l)
            print("-----------------------------------------")

confusion_matrix for nbc with small test data size:
	 P 		 N 
 T 	 0 		 0 
 F 	 0 		 0 


confusion_matrix for nbc with large test data size:
	 P 		 N 
 T 	 0 		 0 
 F 	 0 		 0 

-----------------------------------------
confusion_matrix for svc with small test data size:
	 P 		 N 
 T 	 0 		 0 
 F 	 0 		 0 


confusion_matrix for svc with large test data size:
	 P 		 N 
 T 	 0 		 0 
 F 	 0 		 0 

-----------------------------------------
confusion_matrix for mlp with small test data size:
	 P 		 N 
 T 	 0 		 0 
 F 	 0 		 0 


confusion_matrix for mlp with large test data size:
	 P 		 N 
 T 	 0 		 0 
 F 	 0 		 0 

-----------------------------------------
confusion_matrix for knn with small test data size:
	 P 		 N 
 T 	 21 		 171 
 F 	 42 		 13 


confusion_matrix for knn with large test data size:
	 P 		 N 
 T 	 30 		 342 
 F 	 86 		 36 

-----------------------------------------
classification_report for nbc with small test data size:
              precision    recall  f1-score   sup